<a href="https://colab.research.google.com/github/antonkravchenko2001/NLP-Telegram-Bot/blob/master/Transformer_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, SpatialDropout1D, Input, Embedding
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import random as python_random
import json

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates
def pos_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class MultiheadSelfAttention(tf.keras.Model):
    def __init__(self, d_model, num_heads, mask = False):
        super(MultiheadSelfAttention, self).__init__()
        self.depth = d_model // num_heads
        self.num_heads = num_heads
        self.query_dense = tf.keras.layers.Dense(d_model)
        self.key_dense = tf.keras.layers.Dense(d_model)
        self.value_dense = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)
        self.mask = mask
    

    def create_mask(self, length, width):
        mask = np.zeros((length, width), dtype='float32')
        for i in range(length):
            for j in range(width):
                if j > i:
                    mask[i,j] = -1e9
        return tf.cast(mask, dtype=tf.float32)


    def call(self,query, key, value):
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        batch_size = query.shape[0]
        query = tf.reshape(query, [batch_size, -1, self.num_heads, self.depth])
        query = tf.transpose(query, [0, 2, 1, 3])
        key = tf.reshape(key, [batch_size, -1, self.num_heads, self.depth])
        key = tf.transpose(key, [0, 2, 1, 3])
        value = tf.reshape(value, [batch_size, -1, self.num_heads, self.depth])
        value = tf.transpose(value, [0, 2, 1, 3])
        score = tf.matmul(query, key, transpose_b=True)
        score /= tf.math.sqrt(tf.dtypes.cast(self.depth, dtype=tf.float32))
        if self.mask:
            score += self.create_mask(score.shape[-2], score.shape[-1])
        alignment = tf.nn.softmax(score, axis=-1)
        context = tf.matmul(alignment, value)
        context = tf.transpose(context, [0, 2, 1, 3])
        context = tf.reshape(context, [batch_size, -1, self.depth * self.num_heads])
        output = self.dense(context)
        # heads has shape (batch, decoder_len, model_size)
        return output

In [ ]:
class Encoder(tf.keras.Model):
  
    def __init__(self, vocab_size, d_model, num_heads,  num_layers=1, dropout=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.dropouts1 = [Dropout(dropout) for i in range(num_layers)]
        self.dropouts2 = [Dropout(dropout) for i in range(num_layers)]
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, d_model)
        self.mha = [MultiheadSelfAttention(d_model, num_heads) for _ in range(num_layers)]
        self.layer_normalization_1 = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.layer_normalization_2 = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.dense_1 = [tf.keras.layers.Dense(d_model * 4, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(d_model) for _ in range(num_layers)]
        
   
    def call(self, encoder_inputs):
        encoder_outputs = self.embedding(encoder_inputs)
        encoder_outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        encoder_outputs += pos_encoding(encoder_outputs.shape[1], encoder_outputs.shape[2])
        for i in range(self.num_layers):
            attention = self.mha[i](query=encoder_outputs, key=encoder_outputs, value=encoder_outputs)
            attention = self.dropouts1[i](attention)
            attention = self.layer_normalization_1[i](encoder_outputs + attention)
            encoder_outputs = self.dense_1[i](attention) 
            encoder_outputs = self.dense_2[i](encoder_outputs)
            encoder_outputs = self.dropouts2[i](encoder_outputs)
            encoder_outputs = self.layer_normalization_2[i](attention + encoder_outputs)
        return encoder_outputs

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, model_size, num_layers, h):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, d_model)
        self.attention = [MultiHeadAttention(d_model, num_heads) for _ in range(num_layers)]
        self.attention_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(d_model) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
    def call(self, sequence):
        embed_out = self.embedding(sequence)
        embed_out += positional_encoding(embed_out)
        sub_in = embed_out
        for i in range(self.num_layers):
            sub_out = self.attention[i](query=sub_in, key=sub_in, value=sub_in)
            sub_out = sub_in + sub_out
            sub_out = self.attention_norm[i](sub_out)
            ffn_in = sub_out
            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_in + ffn_out
            ffn_out = self.ffn_norm[i](ffn_out)
            sub_in = ffn_out
        return ffn_out

In [ ]:
class Decoder(tf.keras.Model):

    def __init__(self, vocab_size, d_model, num_heads, num_layers=1, dropout=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, d_model)
        self.masked_mha = [MultiheadSelfAttention(d_model, num_heads, mask=True) for _ in range(num_layers)]
        self.layer_normalization_1 = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.layer_normalization_2 = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.layer_normalization_3 = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.dropouts1 = [tf.keras.layers.Dropout(dropout) for _ in range(num_layers)]
        self.dropouts2 = [tf.keras.layers.Dropout(dropout) for _ in range(num_layers)]
        self.dropouts3 = [tf.keras.layers.Dropout(dropout) for _ in range(num_layers)]
        self.mha = [MultiheadSelfAttention(d_model, num_heads) for _ in range(num_layers)]
        self.dense_1 = [tf.keras.layers.Dense(d_model * 4, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(d_model) for _ in range(num_layers)]
        self.dense = tf.keras.layers.Dense(vocab_size+1, activation = 'softmax')

    def call(self, encoder_outputs, decoder_inputs):
        decoder_outputs = self.embedding(decoder_inputs)
        decoder_outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        decoder_outputs += pos_encoding(decoder_outputs.shape[1], decoder_outputs.shape[2])
        for i in range(self.num_layers):
            masked_attention = self.masked_mha[i](query=decoder_outputs, key=decoder_outputs, value=decoder_outputs)
            masked_attention = self.dropouts1[i](masked_attention)
            masked_attention = self.layer_normalization_1[i](masked_attention + decoder_outputs)
            attention = self.mha[i](query=masked_attention, key=encoder_outputs, value=encoder_outputs)
            attention = self.dropouts2[i](attention)
            attention = self.layer_normalization_2[i](attention+masked_attention)
            decoder_outputs = self.dense_1[i](attention)
            decoder_outputs = self.dense_2[i](decoder_outputs)
            decoder_outputs = self.dropouts3[i](decoder_outputs)
            decoder_outputs = self.layer_normalization_3[i](decoder_outputs + attention)
        decoder_outputs = self.dense(decoder_outputs)
        return decoder_outputs

In [ ]:
class Transformer(tf.keras.Model):
    
    def __init__(self, input_vocab_size, target_vocab_size, d_model, num_heads, num_layers_encoder=1, num_layers_decoder=1):
        super().__init__()
        self.encoder = Encoder(vocab_size=input_vocab_size, d_model=d_model, num_heads=num_heads, num_layers=num_layers_encoder)
        self.decoder = Decoder(vocab_size=target_vocab_size,d_model=d_model, num_heads=num_heads, num_layers=num_layers_decoder)
    
    def call(self, inputs):
        encoder_outputs = self.encoder(inputs[0])
        output = self.decoder(encoder_outputs, inputs[1])
        return output

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
def conv_dict(url):
    conv_dict = {}
    file = open(url, mode='r',encoding='unicode_escape')
    for line in file:
        line = line.split(' +++$+++ ')
        conv_dict[line[0]] = line[4]
    return conv_dict

In [ ]:
def parse(url):
    file = open(url,mode='r',encoding='unicode_escape')
    inputs = []
    targets = []
    c = 0
    for line in file:
        line = line.split(' +++$+++ ')[3]
        line = line.replace("[","").replace("]","").replace(", ",",").replace("'","").replace("\n","").replace("\t","")
        line = line.split(",")
        for i in range(len(line)-1):
            if len(d[line[i]].split()) < 25 and len(d[line[i+1]].split()) <25:
                inputs.append(d[line[i]] )
                targets.append("start_token " + d[line[i+1]] + " end_token")
    return inputs,targets

In [ ]:
def tokenize(text):
    tokenizer = Tokenizer(num_words=20000, filters='!"#$%&()*+,-./:;<=>?@[\\]^{|}`~\t\n')
    tokenizer.fit_on_texts(text)
    text = tokenizer.texts_to_sequences(text)
    return text, tokenizer.word_index, {v: k for k, v in tokenizer.word_index.items()}

In [ ]:
def max_sequence_length(text):
    length_list = []
    for sentence in text:
        l = 0
        for _ in sentence:
            l += 1
        length_list.append(l)
    return max(length_list)

In [ ]:
def min_sequence_length(text):
    length_list = []
    for sentence in text:
        l = 0
        for _ in sentence:
            l += 1
        length_list.append(l)
    return min(length_list)

In [ ]:
def batch_generator(x, y, batch_size=128):
    for i in range(0, len(x), batch_size):
        encoder_inputs = pad_sequences(x[i:i+batch_size], maxlen=max_sequence_length(input_text), padding='post')
        decoder_inputs = []
        decoder_outputs = []
        for j, txt in enumerate(y[i:i+batch_size]):
            decoder_inputs.append(txt[:-1])
            decoder_outputs.append(txt[1:])
        decoder_inputs = pad_sequences(decoder_inputs,maxlen=max_sequence_length(target_text)-1, padding='post')
        decoder_outputs = pad_sequences(decoder_outputs,maxlen=max_sequence_length(target_text)-1,padding='post')
        yield ([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
d = conv_dict('/content/drive/My Drive/transformer_model/transformer_data/movie_lines.txt')
input_text, target_text = parse('/content/drive/My Drive/transformer_model/transformer_data/movie_conversations.txt')
input_text = input_text[:102000]
target_text = target_text[:102000]

In [ ]:
input_text, input_dict, input_dict_r = tokenize(input_text)
target_text, target_dict, target_dict_r = tokenize(target_text)

In [ ]:
with open('/content/drive/My Drive/transformer_model/input_dict.json', 'w') as fp:
    json.dump(input_dict, fp, indent=4)
with open('/content/drive/My Drive/transformer_model/target_dict.json', 'w') as fp:
    json.dump(target_dict, fp, indent=4)
with open('/content/drive/My Drive/transformer_model/target_dict_r.json', 'w') as fp:
    json.dump(target_dict_r, fp, sort_keys=True,indent=4)

In [ ]:
for inp, tar in zip(input_text[:], target_text[:]):
    if len(inp) > 20  or len(tar) > 20 or len(tar) < 3:
        target_text.remove(tar)
        input_text.remove(inp)

In [ ]:
input_text =input_text[:100000]
target_text = target_text[:100000]

In [ ]:
num_heads = 8
d_model = 256
input_vocab_size=len(input_dict)
target_vocab_size=len(target_dict)
batch_size=100

In [ ]:
transformer = Transformer(input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, d_model=d_model, num_heads=num_heads, num_layers_encoder=1, num_layers_decoder=1)

In [ ]:
target_dict_r[20]

'know'

In [ ]:
transformer.load_weights('/content/drive/My Drive/transformer_model/transformer_data/my_model')

IndexError: ignored

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, model_size, warmup_steps=40000):
    super(CustomSchedule, self).__init__()
    self.model_size = model_size
    self.model_size = tf.cast(self.model_size, tf.float32)
    self.warmup_steps = warmup_steps
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    return tf.math.rsqrt(self.model_size) * tf.math.minimum(arg1, arg2)

In [ ]:
lr = CustomSchedule(d_model)

In [ ]:
optimizer = Adam(0.000004)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
train_loss = tf.keras.metrics.Mean(name='train_loss')
sparse_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
train_accuracy = tf.keras.metrics.Mean()

In [ ]:
checkpoint_path = "/content/drive/My Drive/transformer_model/my_model"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [ ]:
def train_step(x, y):
    with tf.GradientTape() as tape:
        x[0] = tf.cast(x[0], tf.int64)
        x[1] = tf.cast(x[1], tf.int64)
        y_prediction = transformer(x)
        loss = loss_func(y, y_prediction)
        accuracy = sparse_accuracy(y, y_prediction)
    variables = transformer.trainable_variables
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))
    train_loss.update_state(loss)
    train_accuracy.update_state(accuracy)

In [ ]:
for epoch in range(5):
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_data = batch_generator(input_text, target_text, batch_size=400)
    for  i, train_batch  in enumerate(train_data):
        x_train = train_batch[0]
        y_train = train_batch[1]
        train_step(x_train, y_train)
    print ('Epoch {} train_loss {:.4f} train_accuracy {:.4f}'.format(epoch + 1, train_loss.result(), train_accuracy.result()))

Epoch 1 train_loss 0.2914 train_accuracy 0.9425
Epoch 2 train_loss 0.2907 train_accuracy 0.9403
Epoch 3 train_loss 0.2899 train_accuracy 0.9402
Epoch 4 train_loss 0.2892 train_accuracy 0.9403
Epoch 5 train_loss 0.2885 train_accuracy 0.9403


In [ ]:
/content/drive/My Drive/transformer_model/my_model.data-00000-of-00001

In [ ]:
transformer.save_weights('/content/drive/My Drive/transformer_model/my_model', save_format = 'tf')

In [ ]:
def encode(sentence):
    sentence = sentence.split()
    for i, el  in enumerate(sentence):
        sentence[i] = input_dict[el]
    sentence = pad_sequences([sentence], maxlen=max_sequence_length(input_text), padding='post')
    return sentence

In [ ]:
transformer = Transformer(input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, d_model=d_model, num_heads=num_heads, num_layers_encoder=1, num_layers_decoder=1)

In [ ]:
transformer.load_weights('/content/drive/My Drive/transformer_model/my_model')

In [ ]:
transformer.trainable_variables

[<tf.Variable 'transformer_5/encoder_5/embedding_10/embeddings:0' shape=(26968, 256) dtype=float32, numpy=
 array([[-0.00259165, -0.00022645,  0.01178451, ..., -0.06673856,
         -0.00726049, -0.06303571],
        [-0.02505769, -0.01281697, -0.00782532, ..., -0.06481081,
          0.01023852, -0.06831598],
        [-0.00660746, -0.03503629, -0.02288005, ..., -0.05209285,
          0.0408354 , -0.06261665],
        ...,
        [-0.03227764, -0.01007773,  0.02982065, ..., -0.033296  ,
         -0.02320027,  0.03281745],
        [ 0.03495635,  0.02848566, -0.00529525, ..., -0.00966644,
          0.026294  , -0.00065624],
        [ 0.03631881,  0.04476119,  0.02508478, ..., -0.01153406,
          0.03906626, -0.02224472]], dtype=float32)>,
 <tf.Variable 'transformer_5/encoder_5/multihead_self_attention_15/dense_85/kernel:0' shape=(256, 256) dtype=float32, numpy=
 array([[-1.88147813e-01, -2.85924226e-02,  1.34183839e-01, ...,
         -1.33470818e-01, -6.84403703e-02, -1.46766484e-04],

In [ ]:
input_text[5]

[14794, 1083, 14795, 16, 12, 25, 343]

In [ ]:
def predict(sentence):
    enc_input = encode(sentence)
    print(sentence)
    out_words = []
    de_input = tf.constant([[1]], dtype=tf.int64)
    while True:
        de_output = transformer([enc_input, de_input])
        new_word = tf.expand_dims(tf.argmax(de_output, -1)[:, -1], axis=1)
        de_input = tf.concat((de_input, new_word), axis=-1)
        out_words.append(target_dict_r[new_word.numpy()[0][0]])
        if out_words[-1] == 'end_token' or len(out_words) >= 20:
            break
    print(de_input)
    print(' '.join(out_words))

In [ ]:
sentence = "go around"
predict(sentence)

go around
tf.Tensor([[   1    5 1947   16    7  122  772    2]], shape=(1, 8), dtype=int64)
the rate is a little charlie end_token


In [ ]:
transformer.trainable_variables

[<tf.Variable 'transformer_1/encoder_1/embedding_2/embeddings:0' shape=(26968, 256) dtype=float32, numpy=
 array([[-0.00259165, -0.00022645,  0.01178451, ..., -0.06673856,
         -0.00726049, -0.06303571],
        [-0.02505769, -0.01281697, -0.00782532, ..., -0.06481081,
          0.01023852, -0.06831598],
        [-0.00660746, -0.03503629, -0.02288005, ..., -0.05209285,
          0.0408354 , -0.06261665],
        ...,
        [-0.03227764, -0.01007773,  0.02982065, ..., -0.033296  ,
         -0.02320027,  0.03281745],
        [ 0.03495635,  0.02848566, -0.00529525, ..., -0.00966644,
          0.026294  , -0.00065624],
        [ 0.03631881,  0.04476119,  0.02508478, ..., -0.01153406,
          0.03906626, -0.02224472]], dtype=float32)>,
 <tf.Variable 'transformer_1/encoder_1/multihead_self_attention_3/dense_17/kernel:0' shape=(256, 256) dtype=float32, numpy=
 array([[-1.88147813e-01, -2.85924226e-02,  1.34183839e-01, ...,
         -1.33470818e-01, -6.84403703e-02, -1.46766484e-04],
 

In [ ]:
input_text[5]

[14794, 1083, 14795, 16, 12, 25, 343]

In [ ]:
[input_dict_r[14794], input_dict_r[1083], input_dict_r[14795], input_dict_r[16], input_dict_r[12], input_dict_r[25], input_dict_r[343]]

["c'esc", 'ma', 'tete', 'this', 'is', 'my', 'head']

In [ ]:
target_text[20]

[1, 12, 23, 8529, 2103, 10628, 18, 37, 1612, 308, 2]